<a href="https://colab.research.google.com/github/OleksiiLatypov/Regression_of_Used_Car_Prices/blob/main/to_bin_or_not_to_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e9:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76728%2F9057646%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240917%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240917T204145Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db3065a143da210d12640a46d1ff1f2e156f3cac8c326ef9460c3c596dfb9276ec6329be9920dcaa3f36299b5394cf70fbc4a168b9d0903c8b6c990164a3db41497545f7ece75191e46be15816e0d375b29da86d988e209cb4fce085f6144f06c154e1378fdd17cd664a1eecc18969e8721076fe3ebddcabb7b80702f4d4b004dd2adc8e8deef15db81606e0ee1b9c84596a868934b913effb6c610cd002e0b7f45ff1ab40e3c93828e2ff6369bc03dbde92d3f55e2a4c08296e644fa330209e034ac59eb0fce891945bfd5d8275b96a8bbec68d2bc3d67a50422052f79ca4ed25e47877d33d691cee3b51f5dc201d7e9e805b0d8d108ea6b4d80d5c3cc9d92b0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import matplotlib.pyplot as plt
import re
from scipy import stats
from scipy.special import inv_boxcox
from datetime import datetime
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge, SGDRegressor, LinearRegression
import category_encoders as ce
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
sns.displot(np.log(train['price']), kde=True)

Concate

Concatenate train and test datasets along rows to form a single DataFrame for preprocessing.
This approach allows us to apply consistent transformations (e.g., encoding, scaling) to both
train and test data at the same time, ensuring no discrepancies between them.
`ignore_index=True` is used to reset the index and avoid duplicated indices from the original datasets.

In [ ]:
df = pd.concat([train, test], axis=0, ignore_index=True)

Drop the 'id' column from the DataFrame as it is not relevant for the prediction task.

In [ ]:
df = df.drop('id', axis=1)

In [ ]:
# price, price_lambda = stats.boxcox(train['price'])
# print(price_lambda)
# milage, milage_lambda = stats.boxcox(train['milage'])
# print(milage_lambda)
# age, age_lambda = stats.boxcox(datetime.now().year - train['model_year'] + 1)
# print(age_lambda)

Check for duplicates

In [ ]:
duplicates = df.duplicated()
duplicates.any()

In [ ]:
train['price'] = price
# df['age'] = age
# df['milage'] = milage
# df.head()

Group the training data by 'brand' and calculate both the mean and median prices for each car brand.
This helps identify the most expensive brands in terms of price, as both metrics provide insights:
- 'mean' shows the average price per brand, which can be skewed by outliers.
- 'median' gives a more robust central tendency, less affected by extreme values.
The results are then sorted by 'median' price in descending order to highlight the top luxury car brands.

In [ ]:
lux_car = train.groupby(['brand'])['price'].agg(['mean', 'median'])
lux_car.sort_values(by='median', ascending=False)

Define price bins and corresponding labels to categorize the cars into different price ranges.
Bins are specified to capture a range from affordable to luxury car prices, with the upper limit covering high-end cars.
The 'pd.cut()' function assigns each car in the 'price' column to one of the categories based on the defined bins.
'include_lowest=True' ensures that the lowest price falls into the first bin.

In [ ]:
price_bins = [0, 15000, 25000, 40000, 70000, 300000]  # Adjust based on your price distribution
price_labels = [0, 1, 3, 4, 5]

# Bin the brands based on their median prices using pd.cut
lux_car['price_category'] = pd.cut(lux_car['median'], bins=price_bins, labels=price_labels, include_lowest=True)


In [ ]:
brand_to_price_category = lux_car['price_category'].to_dict()
df['price_category'] = df['brand'].map(brand_to_price_category)
df['price_category'].value_counts()

In [ ]:
df['age'] = datetime.now().year - df['model_year']
df = df.drop('model_year', axis=1)

In [ ]:
df.head(10)

In [ ]:
lux_auto = [
    "Mercedes-Benz",
    "Audi",
    "BMW",
    "Tesla",
    "Cadillac",
    "Land Rover",
    "Porsche",
    "Lexus",
    'Genesis',
    "INFINITI",
    "Jaguar",
    "Rolls-Royce",
    "Bentley",
    "Ferrari",
    "Aston Martin",
    "Lamborghini",
    "Bugatti",
    "Maserati",
    "Lucid",
    "Polestar",
    "Maybach",
    "Lincoln"
]

In [ ]:
df['premium_auto'] = df['brand'].apply(lambda x: 1 if x in lux_auto else 0)
df.head()

In [ ]:
num = df.iloc[:188533, :].select_dtypes(include=['number'])
correlation = num.corr()
sns.heatmap(correlation, annot=True)

In [ ]:
horsepower = []
engine_volume = []
num_of_cylinders = []
for col in df['engine']:
    hp = re.search(r'(\d+\.?\d*)HP', col, re.IGNORECASE)
    #print(hp)
    if hp:
        horsepower.append(float(hp.group(1)))
    else:
        horsepower.append(0)

    volume = re.search(r'(\d+\.?\d*)L|(\d+\.?\d*\s*)Liter', col, re.IGNORECASE)
    if volume:
        if volume.group(1):
            engine_volume.append(float(volume.group(1)))
        elif volume.group(2):
            engine_volume.append(float(volume.group(2)))
    else:
        engine_volume.append(0)

    cylinder = re.search(r'(\d+)\s*Cylinder|V(\d+)|I(\d+)', col, re.IGNORECASE)

    if cylinder:
        # If we find a numeric cylinder, use that
        if cylinder.group(1):
            num_of_cylinders.append(float(cylinder.group(1)))
        # If we find a V-style code, use the number after 'V'
        elif cylinder.group(2):
            num_of_cylinders.append(float(cylinder.group(2)))
        elif cylinder.group(3):
            num_of_cylinders.append(float(cylinder.group(3)))
    else:
        # If no match is found, default to 0
        num_of_cylinders.append(0)

print(len(horsepower))
print(len(engine_volume))
print(len(num_of_cylinders))
df['horsepower'] = horsepower
df['engine_volume'] = engine_volume
df['num_of_cylinders'] = num_of_cylinders


In [ ]:
df[df['num_of_cylinders'] == 0]['engine'].value_counts()

In [ ]:
# Calculate the mode of the 'num_of_cylinders' column
mode_cylinders = df['num_of_cylinders'].mode()[0]
print(mode_cylinders)

# Use a conditional mask to filter rows where 'num_of_cylinders' is 0 and 'engine' does not contain 'Electric'
mask = (df['num_of_cylinders'] == 0) & (~df['engine'].str.contains('Electric'))

# Replace 0 values with the mode for the filtered rows
df.loc[mask, 'num_of_cylinders'] = mode_cylinders

In [ ]:
sns.histplot(df['num_of_cylinders'])

In [ ]:
100 *55440/314223

In [ ]:
to_replace = []
for row in df.loc[df['fuel_type'] == 'not supported', 'engine']:
    fuel = re.search(r'(Gasoline|Diesel|Hybrid|Flex Fuel|Electric|Plug-In Hybrid)', row)
    if fuel.group(1) == 'Flex Fuel':
        to_replace.append('E85 Flex Fuel')
    else:
        to_replace.append(fuel.group(1))
print(to_replace)

In [ ]:
df[df['fuel_type'] == 'not supported'][['fuel_type', 'engine']]

In [ ]:
df.loc[df['fuel_type'] == 'not supported', 'fuel_type'] = to_replace

In [ ]:
df['fuel_type'].value_counts()

In [ ]:
df.loc[df['fuel_type'] == 'not supported', 'fuel_type']

In [ ]:
df['fuel_type'].isna().sum()

In [ ]:
df[pd.isna(df['fuel_type'])][['fuel_type', 'engine']]

In [ ]:
def determine_fuel_type(engine_str):

    if 'Hybrid' in engine_str:
        return 'Hybrid'
    elif 'Gasoline' in engine_str:
        return 'Gasoline'
    elif 'Flex Fuel' in engine_str:
        return 'E85 Flex Fuel'
    elif 'Diesel' in engine_str or 'Dual Motor' in engine_str:
        return 'Diesel'
    else:
        return 'Electric'

In [ ]:
# Find rows where 'fuel_type' is missing
missing_fuel = df[pd.isna(df['fuel_type'])]

# Determine the type of fuel for these rows
missing_fuel['fuel_type'] = missing_fuel['engine'].apply(determine_fuel_type)

# Update the original DataFrame with the new fuel types
df.update(missing_fuel[['fuel_type']])

# Print the number of updated entries
print(len(missing_fuel))

In [ ]:
df['fuel_type'].unique()

In [ ]:
df['fuel_type'] = df['fuel_type'].replace('–', 'Unknown')

In [ ]:
df['transmission'].unique()

In [ ]:
df['price'].max()

In [ ]:
def categorize_transmission(transmission):
    transmission = str(transmission).lower()
    if 'manual' in transmission or 'm/t' in transmission or 'mt' in transmission:
        return 'Manual'
    elif 'automatic' in transmission or 'a/t' in transmission\
        or 'cvt' in transmission or 'variable' in transmission\
        or 'dct' in transmission or 'at/mt' in transmission\
        or 'dual' in transmission or 'at' in transmission:
        return 'Automatic'
    else:
        return 'Unknown'
    return transmission

In [ ]:
#df['transmission_category'] = df['transmission'].apply(categorize_transmission)
df['transmission_category'] = df['transmission'].apply(categorize_transmission)

In [ ]:
def extract_num_speeds(transmission_str):
    # Regular expression to match the number of speeds
    match = re.search(r'(\d+)\s*[-\s]*Speed', transmission_str, re.IGNORECASE)
    if match:
        return int(match.group(1))
    else:
        return 0

In [ ]:
df['num_speed'] = df['transmission'].apply(extract_num_speeds)

In [ ]:
# type_trans = []
# for row in df['transmission']:
#     # Ensure the row is not NaN
#     if pd.notna(row):
#         # Check for various transmission types
#         if any(keyword in row for keyword in ['Automatic', 'A/T', 'AT', 'A/t', 'Transmission Overdrive Switch']):
#             type_trans.append('A')
#         elif 'DCT' in row:
#             type_trans.append('DCT')
#         elif 'Transmission w/Dual Shift Mode' in row:
#             type_trans.append('Transmission w/Dual Shift Mode')
#         elif 'CVT' in row or 'CVT-F' in row:
#             type_trans.append('CVT')
#         elif 'Variable' in row:
#             type_trans.append('Variable')
#         elif 'Single-Speed Fixed Gear' in row:
#             type_trans.append('Single-Speed Fixed Gear')
#         elif row.strip() == 'F':  # Use strip() to avoid false matches for 'F'
#             type_trans.append('F')
#         elif 'At/Mt' in row:
#             type_trans.append('At/Mt')
#         elif '–' in row:  # Handle dash case
#             type_trans.append('Unknown')
#         else:
#             type_trans.append('M')  # Assume manual transmission by default
#     else:
#         # Handle missing or NaN values if any
#         type_trans.append('Unknown')

# # Output the unique transmission types
# print(set(type_trans))
#df['type_transmission'] = type_trans
# df['num_of_speed'] = num_of_speed

In [ ]:
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])
df.isna().sum()

In [ ]:
sns.histplot(df['accident'])

In [ ]:
df = df.copy()
df.head()

In [ ]:
base_colors = df['ext_col'].str.lower().value_counts()[:10].index.tolist()
base_colors

In [ ]:
print("Base colors:", base_colors)

# Function to extract and standardize colors
def extract_colors(colors):
    colors = str(colors).strip().lower()  # Convert to lowercase and strip whitespace
    if colors in base_colors:
        return colors
    return 'other'

# Apply the function to the 'ext_col' column
df['ext_col'] = df['ext_col'].apply(extract_colors)

In [ ]:
base_color_int = df['int_col'].str.lower().value_counts()[:6].index.tolist()
base_color_int

In [ ]:
def extract_int_colors(colors):
    colors = str(colors).lower()
    if colors in base_color_int:
        return colors
    return 'other'

In [ ]:
df['int_col'] = df['int_col'].apply(extract_int_colors)

In [ ]:
df.head()

In [ ]:
df = df.drop(['engine', 'transmission', 'clean_title'], axis=1)

In [ ]:
sns.histplot(df['horsepower'])

In [ ]:
med_hp = round(df['horsepower'].median())
df.loc[df['horsepower'] == 0, 'horsepower'] = round(med_hp)


In [ ]:
df['horsepower'].value_counts() / df.shape[0]

In [ ]:
# #price, price_lambda = stats.boxcox(train['price'])
# print(price_lambda)
# milage, milage_lambda = stats.boxcox(df['milage'])
# print(milage_lambda)
# age, age_lambda = stats.boxcox(datetime.now().year - df['model_year'] + 1)
# print(age_lambda)

In [ ]:
df.head()

In [ ]:
df[df['transmission_category'].str.contains('Unknown')]['transmission'].value_counts()

In [ ]:
model_counts = df['model'].value_counts()
df['model_frequency'] = df['model'].map(model_counts)

In [ ]:
df = df.drop('model', axis=1)

In [ ]:
df.head()

In [ ]:
columns_to_encode = ['brand', 'ext_col', 'int_col', 'fuel_type', 'transmission_category']
df_to_encode = df[columns_to_encode]
df_encoded = pd.get_dummies(df_to_encode, columns=columns_to_encode, dtype='int')

In [ ]:
df = df.drop(columns_to_encode, axis=1)

In [ ]:
df_encoded.head()

In [ ]:
le = LabelEncoder()
df['accident'] = le.fit_transform(df['accident'])

In [ ]:
df.head()

In [ ]:
new_df = pd.concat([df, df_encoded], axis=1)

In [ ]:
new_df.head()

In [ ]:
train_df = new_df.iloc[:188533, :]
test_df = new_df.iloc[188533:, :]

In [ ]:
X = train_df

In [ ]:
y = X['price']
X = X.drop('price', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create the pipeline
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Step 1: MinMax scaling
    ('regressor', LinearRegression())  # Step 2: Linear Regression model
])

# Fit the pipeline to the training data
lr_pipeline.fit(X_train, y_train)

# Make predictions
y_pred = lr_pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 69627.90934249542


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Create the pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Step 1: MinMax scaling
    ('regressor', KNeighborsRegressor())  # Step 2: KNeighborsRegressor model
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# Initialize and train the model
model = RandomForestRegressor(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Root Mean Squared Error: {np.sqrt(mse)}')


Root Mean Squared Error: 78153.26771265175


In [ ]:
y_pred

array([34763.82035714, 16616.35      , 34723.7       , ...,
       22046.9       , 14068.9       , 76737.36      ])

In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

In [ ]:

# Target variable
target = 'price'

# Target encoding for 'brand'
brand_mean = train_df.groupby('brand')[target].mean()
train_df['brand_encoded'] = train_df['brand'].map(brand_mean)


# Target encoding for 'model'
model_mean = train_df.groupby('model')[target].mean()
train_df['model_encoded'] = train_df['model'].map(model_mean)

# Drop original categorical columns if needed
train_df = train_df.drop(columns=['brand', 'model'])

train_df.head()


In [ ]:
# le = LabelEncoder()
# train_df['accident'] = le.fit_transform(train_df['accident'])
# test_df['accident'] = le.transform(test_df['accident'])

In [ ]:
train_df.head()

In [ ]:
# df['brand_model'] = df['brand'] + '_' + df['model']
# df = pd.get_dummies(df, columns=['brand_model'])

In [ ]:
X = train_df.copy()

In [ ]:
y = X['price']
X = X.drop('price', axis=1)

In [ ]:
X = X.drop('price', axis=1)

In [ ]:
y = np.log(y)

In [ ]:
# cbe_encoder = ce.cat_boost.CatBoostEncoder(a=1)
# X_encoded = cbe_encoder.fit_transform(X, y)

In [ ]:
# X_encoded.head()

In [ ]:
# #for test split 0.3 is the best !!!
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer
from catboost import CatBoostRegressor


clf_catboost = CatBoostRegressor(iterations=700,
                                  learning_rate=0.03,
                                  depth=7,
                                 l2_leaf_reg=3,
                                border_count=254,
                                 verbose=0)  # verbose=0 to suppress output during cross-validation


# Define k-fold cross-validation
n_splits = 5  # number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation and evaluate RMSE
rmse_scores = -cross_val_score(clf_catboost, X, y, scoring='neg_root_mean_squared_error', cv=kf)

# Calculate mean and standard deviation of RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean RMSE: {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE: {std_rmse:.4f}")

clf_catboost.fit(X, y)
#72823 = 700iter
#72718 = 300iter

In [ ]:
feature_importances = clf_catboost.get_feature_importance()
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Display the DataFrame
print(importance_df)

# Plot feature importances
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
test_df = test_df.drop('price', axis=1)

In [ ]:
#test_X_encoded = cbe_encoder.transform(test_df)

In [ ]:
#test_X_encoded.head()

In [ ]:
# Compute target encoding from training data
test_df['brand_encoded'] = test_df['brand'].map(brand_mean)

# Encode 'model' in the test data using the encoding from training data
test_df['model_encoded'] = test_df['model'].map(model_mean)

# For any values in the test data that were not seen in the training data, you may need to handle NaNs
# test_df['brand_encoded'].fillna(test_df['brand_encoded'].mean(), inplace=True)
# test_df['model_encoded'].fillna(test_df['model_encoded'].mean(), inplace=True)
test_df.head()

In [ ]:
# test_df['brand_encoded'].fillna(train_df['brand_encoded'].mean(), inplace=True)
# test_df['model_encoded'].fillna(train_df['model_encoded'].mean(), inplace=True)

In [ ]:
test_df = test_df.drop(['brand', 'model'], axis=1)

In [ ]:
test_df.head(15)

In [ ]:
test_id = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')
predictions = clf_catboost.predict(test_df)
test_id['price'] = np.exp(predictions)
test_id.to_csv('submission_binning_log.csv', index=False)

In [ ]:
y_hat = clf_catboost.predict(X)[:10]
np.exp(y_hat)

In [ ]:
train['price'][:10]

In [ ]:
# price, price_lambda = stats.boxcox(train['price'])
# print(price_lambda)
# milage, milage_lambda = stats.boxcox(train['milage'])
# print(milage_lambda)
# age, age_lambda = stats.boxcox(datetime.now().year - train['model_year'] + 1)
# print(age_lambda)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor

# Define the pipeline with preprocessing and model
columns_to_scale = ['model_year', 'milage']

# Define the preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', MinMaxScaler(), columns_to_scale)  # Apply MinMaxScaler to specified columns
    ],
    remainder='passthrough'  # Leave other columns unchanged
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Apply preprocessing
    ('model', KNeighborsRegressor(n_neighbors=5))  # Using KNeighborsRegressor
])

# Define k-fold cross-validation
n_splits = 5  # number of folds
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Perform cross-validation and evaluate RMSE
rmse_scores = -cross_val_score(pipeline, X_encoded, y, scoring='neg_root_mean_squared_error', cv=kf)

# Calculate mean and standard deviation of RMSE
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean RMSE: {mean_rmse:.4f}")
print(f"Standard Deviation of RMSE: {std_rmse:.4f}")

# Fit the pipeline on the entire dataset
pipeline.fit(X_encoded, y)


In [ ]:
y_hat_knn = pipeline.predict(X_encoded)[:10]
inv_boxcox(y_hat_knn, price_lambda)